In [1]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 5fd22109-7e01-4065-9ca2-9a71dffe7c8e

In [2]:
import Local_NAV as local
import Global_NAV as glob
import Motion_CTR_ as mctrl
import Kalman_FIL as kfil
import math
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import IPython
from IPython.display import display, clear_output
import time



from matplotlib import colors
from Computer_VIS import *

%matplotlib inline

In [3]:
#Global variables
state = 0
obst = [0,0,0,0,0]
motors_speed = [0,0]
not_arrived = True
k = 0
C_conv_toThymio_right = 67.60908181
C_conv_toThymio_left = 67.82946137
L = 104
R = 20
Ts = 0.02 # sampling time


if state == 0:
    # Definition of the references used to map the space
    color_ref1 = cv2.imread('ref1.jpg')
    ref1 = Obj(color_ref1, 50, 2, np.array([[0,0]]))
    color_ref2 = cv2.imread('ref2.jpg')
    ref2 = Obj(color_ref2, 100, 2, np.array([[0,0]]))
    color_ref3 = cv2.imread('ref3.jpg')
    ref3 = Obj(color_ref3, 100, 2, np.array([[0,0]]))
    map = Map(ref1, ref2, ref3)

    # Definition of the objects to recognize in the space
    color_obstacle = cv2.imread('obstacle.jpg')
    obstacle = Obj(color_obstacle, 700, 4)
    color_goal = cv2.imread('goal.jpg')
    goal = Obj(color_goal, 100, 4)
    color_left_mark = cv2.imread('left_mark.jpg')
    left_mark = Obj(color_left_mark, 50, 2)
    color_right_mark = cv2.imread('right_mark.jpg')
    right_mark = Obj(color_right_mark, 50, 2)
    thymio = Robot(right_mark, left_mark)

    # Open the video
    video = cv2.VideoCapture('test3.avi')
    if (video.isOpened()== False): 
        print("Error opening video stream or file")

    # the objects to draw on the map in the matrix format
    objects = [obstacle]
    refs = [ref1, ref2, ref3]
    detected = False
    while not (detected):
        ret, frame = video.read()
        if ret == True:
            if np.size(thymio.localize(frame,map)[1], 0) > 0:
                detected = True
                print('Robot position : ', thymio.localize(frame,map)[0])
                print('Robot direction : ', vector_to_angle(thymio.localize(frame,map)[1]))
    state_estimate_k = np.array([thymio.localize(frame,map)[0,0],thymio.localize(frame,map)[0,1],vector_to_angle(thymio.localize(frame,map)[1])]) 
    state = 1
        
    # Global navigation state
    if state == 1:
        matrix = map.matrix(frame,objects,thymio,goal,refs)
        path = glob.global_path(matrix)
        N = len(path)
        
        while not_arrived == True:
            prox_horizontal = local.read_prox_sensors(node,client)
            obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
            state = local.update_state(state,obst,client)
            x_goal_temp = path[1,k]
            y_goal_temp = path[2,k]
            if k < N-1:
                theta_goal_temp = (math.atan2(x_goal[k+1]-x_goal[k], y_goal[k+1]-y_goal[k]) + np.pi) % (np.pi/2) - np.pi
            v, w = mctrl.control_law(state_estimate_k, x_goal_temp, y_goal_temp, theta_goal_temp, Kv, Kp_alpha, Kp_beta)
            vr, vl = mctrl.convert_velocity2RL(v,w,C_conv_toThymio_right, C_conv_toThymio_left,L,R)
            vr = int(vr)
            vl = int(vl)
            set_motors(vl,vr,node)
            
            if camera_obstructed == 1 or video.isOpened == False:
                z_k_observation_vector = np.array([state_estimate_k[0], state_estimate_k[1], state_estimate_k[2]])
            if camera_obstructed == 0 :
                # Capture frame-by-frame
                ret, frame = video.read()
                if ret == True:
                    if np.size(thymio.localize(frame,map)[1], 0) > 0:
                        print('Robot position : ', thymio.localize(frame,map)[0])
                        print('Robot direction : ', vector_to_angle(thymio.localize(frame,map)[1]))
                        #time.sleep(0.4)
                        clear_output(wait=True)
                        z_k_observation_vector = np.array([thymio.localize(frame,map)[0,0],thymio.localize(frame,map)[0,1],vector_to_angle(thymio.localize(frame,map)[1])]) 
                        [state_estimate_k,_] = kfil.ekf(z_k_observation_vector, state_estimate_k, [v,w] , P_k_minus_1, Ts)
            if np.linalg.norm(state_estimate_k[:2] - np.array([x_goal_temp, y_goal_temp])) < threshold_for_convergence:
                k = k+1
                if k == N-1:
                    not_arrived = False
                    print('The robot has arrived to the final goal')
            break 
            time.sleep(Ts) 
        state = 3                                               


    # Local navigation state
    if state == 2: 
        
        #We get the sensor value
        prox_horizontal = local.read_prox_sensors(node,client)
        
        #We fill the obst list with the sensor value
        obst = [prox_horizontal[0], prox_horizontal[1], prox_horizontal[2], prox_horizontal[3], prox_horizontal[4]]
        
        #We calculate the motor speed to avoid the object -> potential field method
        motors_speed = local.local_navigation(obst)
        
        #We set the motor speed
        mctrl.set_motors(motors_speed[0],motors_speed[1],node)       
        
        #We update the state
        state = local.update_state(state,obst,client)   


Robot position :  [-0.28922092 -5.46318036]
Robot direction :  -1.7653837640900176


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\mean.dispatch.cpp:516: error: (-215:Assertion failed) !_src.empty() in function 'cv::meanStdDev'


In [9]:
motors_speed = local.global_path()
local.set_motors(motors_speed[0],motors_speed[1],node) 
local.stop_motors(node) 
